In [1]:
from collections import defaultdict
from itertools import chain, combinations

class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    #Count Frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None
    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]
    
    # Initialise Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        #Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        treeNode = treeNode.next  
    return condPats, frequency

def mineTree(headerTable, minSup, preFix, freqItemList):
    #Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])]
    # Start with the lowest frequency 
    for item in sortedItemList:  
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        #Construct conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        #Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

itemSetList = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
freqItemSet, rules = fpgrowth(itemSetList, minSupRatio=0.5, minConf=0.5)
print(freqItemSet)
print(rules)

[{'r'}, {'y'}, {'x', 'y'}, {'y', 'z'}, {'x', 'y', 'z'}, {'t'}, {'t', 'y'}, {'x', 't'}, {'x', 't', 'y'}, {'t', 'z'}, {'x', 't', 'z'}, {'t', 'y', 'z'}, {'x', 't', 'y', 'z'}, {'s'}, {'x', 's'}, {'x'}, {'x', 'z'}, {'z'}]
[[{'x'}, {'y'}, 0.75], [{'y'}, {'x'}, 1.0], [{'y'}, {'z'}, 1.0], [{'z'}, {'y'}, 0.6], [{'x'}, {'y', 'z'}, 0.75], [{'y'}, {'x', 'z'}, 1.0], [{'z'}, {'x', 'y'}, 0.6], [{'x', 'y'}, {'z'}, 1.0], [{'x', 'z'}, {'y'}, 1.0], [{'y', 'z'}, {'x'}, 1.0], [{'t'}, {'y'}, 1.0], [{'y'}, {'t'}, 1.0], [{'x'}, {'t'}, 0.75], [{'t'}, {'x'}, 1.0], [{'x'}, {'t', 'y'}, 0.75], [{'t'}, {'x', 'y'}, 1.0], [{'y'}, {'x', 't'}, 1.0], [{'x', 't'}, {'y'}, 1.0], [{'x', 'y'}, {'t'}, 1.0], [{'t', 'y'}, {'x'}, 1.0], [{'t'}, {'z'}, 1.0], [{'z'}, {'t'}, 0.6], [{'x'}, {'t', 'z'}, 0.75], [{'t'}, {'x', 'z'}, 1.0], [{'z'}, {'x', 't'}, 0.6], [{'x', 't'}, {'z'}, 1.0], [{'x', 'z'}, {'t'}, 1.0], [{'t', 'z'}, {'x'}, 1.0], [{'t'}, {'y', 'z'}, 1.0], [{'y'}, {'t', 'z'}, 1.0], [{'z'}, {'t', 'y'}, 0.6], [{'t', 'y'}, {'z'}, 1